In [1]:
from tensorflow import keras
from keras import datasets, layers, models, preprocessing
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [4]:

import tensorflow as tf
import re
import tensorflow_datasets as tfds
import numpy as np
from nltk import word_tokenize
from keras.utils import pad_sequences
import nltk
nltk.download('punkt')
imdb = datasets.imdb
num_words = 20000
max_len = 200
n_words = 10000
dim_embedding = 128
EPOCHS = 40 
BATCH_SIZE = 500 
def load_data():
        # Load data.
        (X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=n_words)

        # Pad sequences with max_len.
        X_train = pad_sequences(X_train, maxlen=max_len)
        X_test = pad_sequences(X_test, maxlen=max_len)
        print(X_train[0][0])
        return (X_train, y_train), (X_test, y_test)
    
def build_model1():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(layers.Dropout(0.3))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model   

def build_model2():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(layers.Dropout(0.3))
    model.add(layers.Bidirectional(layers.LSTM(128))) 
    #model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model3()://наилучшая модель из 8
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding))
    #model.add(layers.Dropout(0.3))
    model.add(layers.Bidirectional(layers.LSTM(32, return_sequences = True)))
    #model.add(layers.LSTM(128,return_sequences=True)) 
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model4():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    #model.add(layers.Dropout(0.3))
    model.add(layers.LSTM(128,activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model5():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    #model.add(layers.Dropout(0.3))
    model.add(layers.Bidirectional(layers.GRU(64,activation='relu')))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model6():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    #model.add(layers.Dropout(0.3))
    model.add(layers.GRU(128,activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  


def build_model7():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(TransformerEncoder(dim_embedding, 128, 2))
    #model.add(layers.Dropout(0.3))
    model.add(layers.GRU(128,activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model8():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(TransformerEncoder(dim_embedding, 128, 2))
    #model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(128,kernel_size=2,activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model8():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(TransformerEncoder(dim_embedding, 128, 2))
    #model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(128,kernel_size=2,activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

(X_train, y_train), (X_test, y_test) = load_data()
model = build_model3()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy"]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb9.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stasy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


5


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 627ms/step - accuracy: 0.6176 - loss: 0.6652 - val_accuracy: 0.8548 - val_loss: 0.3490
Epoch 2/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 29s 584ms/step - accuracy: 0.8824 - loss: 0.2970 - val_accuracy: 0.8638 - val_loss: 0.3146
Epoch 3/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 647ms/step - accuracy: 0.9236 - loss: 0.2100 - val_accuracy: 0.8706 - val_loss: 0.3197
Epoch 4/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 599ms/step - accuracy: 0.9470 - loss: 0.1535 - val_accuracy: 0.8639 - val_loss: 0.3524
Epoch 5/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 634ms/step - accuracy: 0.9612 - loss: 0.1225 - val_accuracy: 0.8621 - val_loss: 0.3905
Epoch 6/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 756ms/step - accuracy: 0.9729 - loss: 0.0923 - val_accuracy: 0.8566 - val_loss: 0.4205
Epoch 7/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 679ms/step - accuracy: 0.9747 - loss: 0.0843 - val_accuracy: 0.8610 - val_loss: 0.4200
Epoch 8/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 675ms/step - accuracy: 0.9800 - loss: 0.0695 - val_accu


Test score: 1.2705284357070923
Test accuracy: 0.8509600162506104


ValueError: Unable to synchronously create dataset (name already exists)